In [1]:
import gym
import numpy as np
import mjx
from mjx.agents import RandomAgent, ShantenAgent

In [2]:
import gym
import numpy as np
import mjx
from mjx.agents import RandomAgent, ShantenAgent  # 使用 Shanten agent

class GymEnv(gym.Env):
    def __init__(
        self, 
        opponent_agents: list, 
        reward_type: str = "game_tenhou_7dan", 
        done_type: str = "game", 
        feature_type: str = "mjx-small-v0"
    ):
        super(GymEnv, self).__init__()
        self.opponent_agents = opponent_agents  # 对手列表，包含 Shanten agent
        self.reward_type = reward_type
        self.done_type = done_type
        self.feature_type = feature_type

        self.target_player = "player_0"  # 我方玩家
        self.mjx_env = mjx.MjxEnv()
        self.curr_obs_dict = self.mjx_env.reset()

    def reset(self):
        """环境重置，返回第一个状态和可用动作"""
        self.curr_obs_dict = self.mjx_env.reset()

        # 跳过其他玩家的回合，直到轮到我们的玩家
        while self.target_player not in self.curr_obs_dict:
            action_dict = {
                player_id: self.opponent_agents[i].act(obs)
                for i, (player_id, obs) in enumerate(self.curr_obs_dict.items())
            }
            self.curr_obs_dict = self.mjx_env.step(action_dict)

        # 返回当前玩家的特征
        obs = self.curr_obs_dict[self.target_player]
        feat = obs.to_features(self.feature_type)
        mask = obs.action_mask()  # 获取有效动作
        return feat, {"action_mask": mask}

    def step(self, action):
        """执行一个步骤，并返回下一个状态、奖励、是否结束"""
        action_dict = {self.target_player: mjx.Action.select_from(action, self.curr_obs_dict[self.target_player].legal_actions())}

        # 遍历当前所有玩家
        for i, (pid, obs) in enumerate(self.curr_obs_dict.items()):
            if pid != self.target_player:
                # 需要将 i-1 映射到对手代理列表
                action_dict[pid] = self.opponent_agents[i-1].act(obs)

        # 更新状态
        self.curr_obs_dict = self.mjx_env.step(action_dict)

        # 检查是否是当前玩家的回合
        while self.target_player not in self.curr_obs_dict:
            action_dict = {
                player_id: self.opponent_agents[i-1].act(obs)
                for i, (player_id, obs) in enumerate(self.curr_obs_dict.items())
                if player_id != self.target_player
            }
            self.curr_obs_dict = self.mjx_env.step(action_dict)

            if self.mjx_env.done(self.done_type):
                obs = list(self.curr_obs_dict.values())[0]  # 获取最后一个观察
                feat = obs.to_features(self.feature_type)
                done = True
                reward = self.mjx_env.rewards(self.reward_type)[self.target_player]
                return feat, reward, done, {"action_mask": np.ones(181)}  # 动作掩码为 1（游戏结束）

        # 处理游戏继续的情况
        assert self.target_player in self.curr_obs_dict
        obs = self.curr_obs_dict[self.target_player]
        done = self.mjx_env.done(self.done_type)
        reward = self.mjx_env.rewards(self.reward_type)[self.target_player]
        feat = obs.to_features(self.feature_type)
        mask = obs.action_mask()

        return feat, reward, done, {"action_mask": mask}



In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Categorical

class ActorCritic(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(ActorCritic, self).__init__()
        
        self.base = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
        )
        
        self.actor = nn.Linear(hidden_dim, output_dim)
        self.critic = nn.Linear(hidden_dim, 1)
    
    def forward(self, x):
        base_output = self.base(x)
        action_logits = self.actor(base_output)
        state_values = self.critic(base_output)
        return action_logits, state_values

class PPOAgent:
    def __init__(self, input_dim, hidden_dim, output_dim, lr=0.01, gamma=0.99, clip_ratio=0.4, value_coef=0.5, entropy_coef=0.01):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = ActorCritic(input_dim, hidden_dim, output_dim).to(self.device)
        self.optimizer = optim.Adam(self.model.parameters(), lr=lr)
        
        self.gamma = gamma
        self.clip_ratio = clip_ratio
        self.value_coef = value_coef
        self.entropy_coef = entropy_coef
        
        # 用来收集每个 episode 的数据
        self.states = []
        self.action_masks = []
        self.actions = []
        self.rewards = []
        self.values = []
        self.log_probs = []
    
    def act(self, state, action_mask):
        state = torch.FloatTensor(state).flatten().to(self.device)
        mask = torch.FloatTensor(action_mask).to(self.device)
        
        with torch.no_grad():
            action_logits, state_value = self.model(state)
            action_logits = action_logits - (1 - mask) * 1e9
            
            dist = Categorical(logits=action_logits)
            action = dist.sample()
            log_prob = dist.log_prob(action)
        
        self.states.append(state)
        self.action_masks.append(mask)
        self.actions.append(action)
        self.values.append(state_value)
        self.log_probs.append(log_prob)
        
        return action.item()
    
    def store_reward(self, reward):
        self.rewards.append(reward)
    
    def update(self, next_state=None, done=True):
        # 计算 returns 和 advantages
        if not done and next_state is not None:
            next_state = torch.FloatTensor(next_state).flatten().to(self.device)
            with torch.no_grad():
                _, next_value = self.model(next_state)
            last_value = next_value.item()
        else:
            last_value = 0
        
        states = torch.stack(self.states)
        action_masks = torch.stack(self.action_masks)
        actions = torch.stack(self.actions)
        old_log_probs = torch.stack(self.log_probs)
        old_values = torch.cat(self.values)
        
        returns = []
        advantages = []
        R = last_value
        for r in reversed(self.rewards):
            R = r + self.gamma * R
            returns.insert(0, R)
        
        returns = torch.FloatTensor(returns).to(self.device)
        advantages = returns - old_values.detach()
        
        # Normalize advantages
        if len(advantages) > 1:
            advantages = (advantages - advantages.mean()) / (advantages.std() + 1e-8)
        
        action_logits, state_values = self.model(states)
        
        for i in range(len(action_logits)):
            action_logits[i] = action_logits[i] - (1 - action_masks[i]) * 1e9
        
        dist = Categorical(logits=action_logits)
        new_log_probs = dist.log_prob(actions)
        entropy = dist.entropy().mean()
        
        ratios = torch.exp(new_log_probs - old_log_probs.detach())
        surr1 = ratios * advantages
        surr2 = torch.clamp(ratios, 1.0 - self.clip_ratio, 1.0 + self.clip_ratio) * advantages
        actor_loss = -torch.min(surr1, surr2).mean()
        
        value_loss = nn.MSELoss()(state_values.squeeze(), returns)
        loss = actor_loss + self.value_coef * value_loss - self.entropy_coef * entropy
        
        self.optimizer.zero_grad()
        loss.backward()
        nn.utils.clip_grad_norm_(self.model.parameters(), 0.5)
        self.optimizer.step()
        
        # 清空 episode 数据
        self.states = []
        self.action_masks = []
        self.actions = []
        self.rewards = []
        self.values = []
        self.log_probs = []
        
        return {
            'actor_loss': actor_loss.item(),
            'value_loss': value_loss.item(),
            'entropy': entropy.item(),
            'total_loss': loss.item()
        }


In [4]:
# 初始化环境（Shanten agent 作为对手）
opponents = [ShantenAgent() for _ in range(3)]  # 假设我们的 agent 是 player_0，剩下三个是对手
env = GymEnv(opponent_agents=opponents)

# 获取 observation shape 和 action space
obs, info = env.reset()
obs_shape = obs.flatten().shape[0]
action_dim = len(info["action_mask"])  # 动作数量（181）

# 初始化 PPO agent
agent = PPOAgent(
    input_dim=obs_shape,
    hidden_dim=128,
    output_dim=action_dim
)


In [5]:
import matplotlib.pyplot as plt

def plot_rewards(rewards, path="logs/reward_curve.png"):
    plt.figure()
    plt.plot(rewards)
    plt.xlabel("Episode")
    plt.ylabel("Reward")
    plt.title("Training Reward Curve")
    plt.savefig(path)
    plt.close()


In [ ]:
import json

# 训练超参数
num_episodes = 3000
log_interval = 100  # 每 N 个 episode 记录一次
rolling_rewards = []

all_rewards = []
all_actor_loss = []
all_value_loss = []
best_reward = -float("inf")

for episode in range(1, num_episodes + 1):
    obs, info = env.reset()
    total_reward = 0
    done = False

    while not done:
        action = agent.act(obs, info["action_mask"])
        next_obs, reward, done, info = env.step(action)
        agent.store_reward(reward)
        obs = next_obs
        total_reward += reward
    
    stats = agent.update()
    all_rewards.append(total_reward)
    all_actor_loss.append(stats['actor_loss'])
    all_value_loss.append(stats['value_loss'])

    # 打印日志
    if episode % log_interval == 0:
        avg_reward = np.mean(all_rewards[-log_interval:])
        rolling_rewards.append(avg_reward)
        print(f"Episode {episode}/{num_episodes}, avg reward: {avg_reward:.3f}")
        # 更新最佳模型
        if avg_reward > best_reward:
            best_reward = avg_reward
            torch.save(agent.model.state_dict(), "logs/ppo/ppo_model.pt")
            print(f"Best model saved with reward: {best_reward:.3f}")

        log_data = {
            'episode': episode,
            'avg_reward': avg_reward,
            'actor_loss': stats['actor_loss'],
            'value_loss': stats['value_loss'],
            'entropy': stats['entropy'],
            'total_loss': stats['total_loss']
        }
        with open('logs/ppo/training_log.json', 'a') as f:
            json.dump(log_data, f)
            f.write('\n')
# 绘制训练曲线
plot_rewards(rolling_rewards, path=f"logs/ppo/reward_curve_{episode}.png")




Episode 100/3000, avg reward: -127.350
Best model saved with reward: -127.350
Episode 200/3000, avg reward: -128.250
Episode 300/3000, avg reward: -125.550
Best model saved with reward: -125.550
Episode 400/3000, avg reward: -129.600
Episode 500/3000, avg reward: -122.850
Best model saved with reward: -122.850
Episode 600/3000, avg reward: -120.150
Best model saved with reward: -120.150
Episode 700/3000, avg reward: -132.300
Episode 800/3000, avg reward: -122.400
Episode 900/3000, avg reward: -122.400
Episode 1000/3000, avg reward: -130.950
Episode 1100/3000, avg reward: -123.750
Episode 1200/3000, avg reward: -126.900
Episode 1300/3000, avg reward: -124.200
Episode 1400/3000, avg reward: -128.250
Episode 1500/3000, avg reward: -127.800
Episode 1600/3000, avg reward: -123.750
Episode 1700/3000, avg reward: -123.750
Episode 1800/3000, avg reward: -125.100
Episode 1900/3000, avg reward: -123.750
Episode 2000/3000, avg reward: -129.150
Episode 2100/3000, avg reward: -130.950
Episode 2200/

In [7]:
best_reward

-120.15